In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize Spark
spark = SparkSession.builder \
    .appName("TraffyDocker") \
    .getOrCreate()

In [2]:
# ---------------------------------------------------------
# LOAD DATA (Bronze Layer)
# ---------------------------------------------------------

raw_path = "/home/jovyan/work/data/raw/bangkok_traffy_raw.csv"
geo_path = "/home/jovyan/work/data/source/thailand_geography.csv"

df_traffy = spark.read \
    .option("header", "true") \
    .option("inferSchema", "false") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .csv(raw_path)

df_geo = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .csv(geo_path)

In [4]:
# ---------------------------------------------------------
# STEP 1: IMPROVED COORDINATE IMPUTATION
# ---------------------------------------------------------

# Clean Geo Data first
df_geo_clean = df_geo.select(
    trim(col("district")).alias("geo_district"), 
    col("latitude").alias("geo_lat"),       
    col("longitude").alias("geo_long")      
).dropDuplicates(["geo_district"])

# Parse Traffy Data
# We replace square brackets just in case the format is [10.1, 20.2]
df_parsed = df_traffy \
    .withColumn("coords_clean", regexp_replace(col("coords"), r"[\[\]]", "")) \
    .withColumn("long_str", split(col("coords_clean"), ",").getItem(0)) \
    .withColumn("lat_str", split(col("coords_clean"), ",").getItem(1)) \
    .withColumn("raw_long", trim(col("long_str")).cast(DoubleType())) \
    .withColumn("raw_lat", trim(col("lat_str")).cast(DoubleType())) 

# Join
df_joined = df_parsed.join(df_geo_clean, trim(df_parsed.district) == df_geo_clean.geo_district, "left")

df_loc_fixed = df_joined.withColumn("latitude", coalesce(col("raw_lat"), col("geo_lat"))) \
                        .withColumn("longitude", coalesce(col("raw_long"), col("geo_long")))

# Check how many Nulls exist now
print("Rows with NULL latitude:", df_loc_fixed.filter(col("latitude").isNull()).count())

Rows with NULL latitude: 0


In [5]:
df_joined.count()

778255

In [6]:
# ---------------------------------------------------------
# STEP 2: TEMPORAL FEATURES
# ---------------------------------------------------------
df_timestamp = df_loc_fixed \
    .withColumn("timestamp_dt", to_timestamp(substring(col("timestamp"), 1, 19), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("last_activity_dt", to_timestamp(substring(col("last_activity"), 1, 19), "yyyy-MM-dd HH:mm:ss")) \

# ---------------------------------------------------------
# STEP 3: Clean Comment
# ---------------------------------------------------------
df_timestamp = df_timestamp \
    .withColumn("comment", regexp_replace(col("comment"), r"[\n\r\t]", " ")) \
    .withColumn("comment", trim(regexp_replace(col("comment"), r"\s+", " "))) \

# ---------------------------------------------------------
# STEP 4: Convert Rating to Integer
# ---------------------------------------------------------
df_rating = df_timestamp.withColumn("star", col("star").cast(IntegerType()))

In [7]:
# ---------------------------------------------------------
# SELECT FINAL COLUMNS
# ---------------------------------------------------------

output_df = df_rating.select(
    "ticket_id",
    "type", 
    "organization",
    "comment",  
    "photo",
    "photo_after",
    "latitude",          
    "longitude",   
    "address",   
    "province",       
    "district",
    "subdistrict",             
    "state",
    "star",
    "count_reopen",
    "timestamp_dt",
    "last_activity_dt",    
)

In [ ]:
import os
import shutil
import glob

# ---------------------------------------------------------
# EXPORT STRATEGY: SPARK -> PARQUET -> PANDAS -> CSV
# ---------------------------------------------------------

# โฟลเดอร์ชั่วคราว (ให้ Spark เขียนลงที่นี่ก่อน)
temp_folder = "/home/jovyan/work/data/cleansed/temp_output"

# ชื่อไฟล์ปลายทางที่คุณต้องการจริงๆ
final_file = "/home/jovyan/work/data/cleansed/bangkok_traffy_clean.csv"

output_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .option("quoteAll", "true") \
    .option("escape", "\"") \
    .option("emptyValue", "") \
    .option("nullValue", "") \
    .csv(temp_folder)

print("🔄 Renaming file...")

found_files = glob.glob(f"{temp_folder}/part-*.csv")

if not found_files:
    print("❌ Error: ไม่เจอไฟล์ CSV ที่ Spark สร้างไว้")
else:

    source_file = found_files[0]
    
    if os.path.exists(final_file):
        os.remove(final_file)
        
    os.rename(source_file, final_file)
    
    shutil.rmtree(temp_folder)
    
    print(f"✅ Export Successful! File saved at: {final_file}")

🔄 Renaming file...
✅ Export Successful! File saved at: /home/jovyan/work/data/cleansed/bangkok_traffy_clean.csv
